In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/My Drive/BT4222 Data/Data_4 (CNN + Most_Listing_Features + SVD).csv')

In [4]:
df.drop(columns=["reviewer_id","listing_id"],inplace=True)

In [5]:
df.columns

Index(['preference', 'item_factor_0', 'user_factor_0', 'item_factor_1',
       'user_factor_1', 'item_factor_2', 'user_factor_2', 'item_factor_3',
       'user_factor_3', 'item_factor_4',
       ...
       'Air conditioning', 'Kitchen', 'Parking', 'Outdoor', 'Kids',
       'Pets allowed', 'Workspace', 'Gym', 'Breakfast',
       'availability_average'],
      dtype='object', length=104)

In [6]:
y = df.preference

In [8]:
X = df.drop("preference",axis = 1)

In [9]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [10]:
df.head()

,preference,item_factor_0,user_factor_0,item_factor_1,user_factor_1,item_factor_2,user_factor_2,item_factor_3,user_factor_3,item_factor_4,user_factor_4,item_factor_5,user_factor_5,item_factor_6,user_factor_6,item_factor_7,user_factor_7,item_factor_8,user_factor_8,item_factor_9,user_factor_9,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,principal component 11,principal component 12,principal component 13,principal component 14,principal component 15,principal component 16,principal component 17,principal component 18,principal component 19,...,offline_government_id,selfie,government_id,identity_manual,work_email,manual_online,manual_offline,x0_Central Region,x0_East Region,x0_North Region,x0_North-East Region,x0_West Region,x0_apartment,x0_bed and breakfast,x0_boat,x0_boutique hotel,x0_bungalow,x0_condominium,x0_guest suite,x0_hostel,x0_hotel,x0_house,x0_loft,x0_other,x0_townhouse,x0_Entire home/apt,x0_Hotel room,x0_Private room,x0_Shared room,Wifi,Air conditioning,Kitchen,Parking,Outdoor,Kids,Pets allowed,Workspace,Gym,Breakfast,availability_average
0,0.916301,-0.080090,-0.045423,-0.072907,0.119669,0.077274,0.045258,0.037663,0.085235,-0.038879,0.009194,0.038809,-0.038233,0.030629,-0.072737,0.032604,0.082893,0.287333,-0.098187,0.089866,0.021846,0.457398,0.079587,-0.064634,0.153994,-0.051934,-0.065736,0.144516,-0.008217,-0.063698,-0.129827,0.049170,-0.001383,0.098077,-0.016037,0.116498,0.222798,0.063980,0.054217,0.025583,...,1,1,1,1,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,0,0,0,0,0,0,0,0,1.0
1,0.919923,0.081554,-0.049855,-0.105953,0.044933,-0.116901,0.131527,-0.017028,0.111009,-0.086785,-0.002848,-0.020847,0.074678,-0.037473,-0.052219,-0.075626,-0.012367,0.034406,0.001945,0.085947,0.008726,-0.119754,-0.180501,-0.086900,0.045732,-0.066487,0.003221,-0.028782,0.155485,-0.050831,0.132260,0.081763,0.156268,0.005794,0.034003,0.102278,0.059773,0.059134,0.076569,0.007578,...,1,1,1,0,1,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,1,0,0,0,0,0,1,0,1.0
2,0.938573,0.081554,-0.045758,-0.105953,-0.057649,-0.116901,-0.014033,-0.017028,0.041884,-0.086785,0.181634,-0.020847,-0.046492,-0.037473,-0.133720,-0.075626,-0.118024,0.034406,0.067946,0.085947,-0.158080,-0.119754,-0.180507,-0.086903,0.045733,-0.066486,0.003226,-0.028776,0.155484,-0.050832,0.132259,0.081780,0.156259,0.005801,0.034014,0.102272,0.059779,0.059130,0.076567,0.007577,...,1,1,1,0,1,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,1,0,0,0,0,0,1,0,1.0
3,0.926675,0.081554,0.028973,-0.105953,0.145273,-0.116901,0.066612,-0.017028,-0.063751,-0.086785,0.176873,-0.020847,-0.142651,-0.037473,0.024285,-0.075626,0.007321,0.034406,0.025454,0.085947,0.055499,-0.119754,-0.180505,-0.086901,0.045732,-0.066488,0.003220,-0.028781,0.155479,-0.050833,0.132254,0.081782,0.156268,0.005795,0.034015,0.102278,0.059784,0.059132,0.076567,0.007575,...,1,1,1,0,1,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,1,0,0,0,0,0,1,0,1.0
4,0.910008,0.081554,-0.055366,-0.105953,-0.037461,-0.116901,0.030717,-0.017028,0.059124,-0.086785,0.139225,-0.020847,0.046353,-0.037473,-0.032285,-0.075626,-0.020036,0.034406,0.008693,0.085947,-0.050897,-0.119754,-0.180506,-0.086901,0.045737,-0.066488,0.003217,-0.028781,0.155476,-0.050829,0.132253,0.081782,0.156269,0.005795,0.034014,0.102278,0.059783,0.059135,0.076567,0.007575,...,1,1,1,0,1,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,1,0,0,0,0,0,1,0,1.0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
import numpy as np

def mse(y_predicted, y_true):
  return np.mean(np.square(y_predicted - y_true))
mse_training = mse(lr.predict(X_train), y_train)
mse_testing = mse(lr.predict(X_test), y_test)
print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.02359012719202204, testing mse: 0.023685295200281207
training rmse: 0.15359077834304388, testing rmse: 0.15390027680378357


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [15]:
forest = RandomForestRegressor(random_state = 1, n_estimators = 10, min_samples_split = 2)
params = {
            'max_depth': [5, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 5]
        }

forest = GridSearchCV(forest, params, cv = 3, verbose = 1, n_jobs = -1)
forest.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 19.2min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=10, n_jobs=None,
                                             oob_score=False, random_state=1,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=-

In [16]:
print('best parameters: ', forest.best_params_)

best parameters:  {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}


In [17]:
y_train_pred = forest.predict(X_train)
y_pred = forest.predict(X_test)
mse_training = mse(forest.predict(X_train), y_train)
mse_testing = mse(forest.predict(X_test), y_test)
print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.019791583904796642, testing mse: 0.021824366659435622
training rmse: 0.14068256432407195, testing rmse: 0.14773072347834632


In [18]:
import matplotlib.pyplot as plt
plt.grid_search(forest.cv_results_, change='max_depth', kind='bar')

AttributeError: ignored

In [19]:
from xgboost import XGBRegressor

In [20]:
import numpy as np

In [21]:
xgb1 = XGBRegressor()
parameters = {
              'objective':['reg:linear','reg:squarederror'],
              'learning_rate': [.03, 0.05], #so called `eta` value
              'max_depth': [5, 6, 7],
              'n_estimators': [10,20]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  48 out of  48 | elapsed:  3.5min finished


[13:28:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=2, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=5,
             param_grid={'learning_rate': [0.03, 0.05], 'max_depth': [5, 6, 7],
                         'n_estimators': [10, 20],
    

In [22]:
y_train_pred = xgb_grid.predict(X_train)
y_pred = xgb_grid.predict(X_test)
mse_training = mse(xgb_grid.predict(X_train), y_train)
mse_testing = mse(xgb_grid.predict(X_test), y_test)
print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.03592882099679189, testing mse: 0.03665097320608705
training rmse: 0.18954899365808275, testing rmse: 0.19144443895315175
